## Welcome to the basic neural network tutorial!
The following notebook is designed to walk through the process of building and training your first neural network using the standalone MaCh3 python utilities!

If you're writing scripts this is encapsulate in the objects in `MaCh3PythonUtils/config_reader` however this guide aims to break apart the process of writing the code for yourself!

The first step is to load in a MaCh3 MCMC fit as input file. This is done using the ChainHandler class!

In [ ]:
# MaCh3Python Deps
from MaCh3PythonUtils.file_handling.chain_handler import ChainHandler
from MaCh3PythonUtils.machine_learning.ml_factory import MLFactory

# Other imports
from matplotlib import pyplot as plt
from pathlib import Path
import gdown

In [ ]:
# Download file from google drive

file_url="https://drive.google.com/file/d/1iE6xFhn3BH_HnLUfQ7KFGy2wfeH52Rwf/view?usp=sharing"

# download the file
input_file = Path("../models/demo_chain.root")

if not input_file.exists():
    # download the file
    input_file.parent.mkdir(parents=True, exist_ok=True)
    gdown.download(file_url, str(input_file), quiet=False, fuzzy=True)


In [ ]:
# Load in a file, for the purposes of this example we'll use a
input_file = Path("../../../../T2K/AdaptiveTune/SummerProj/Adapt/NOvA/LongFit/Cut/mcmc_Adapt_NOvA_all_on.root")

# Set up file properties
chain_name = 'posteriors'
verbose=False

chain_handler = ChainHandler(str(input_file), chain_name, verbose=verbose)

Okay, now we've loaded in a file we need to do a bit of processing. This means we need to select the variables we care about and apply parameter cuts

In [ ]:
# Firstly we set things we want to ignore, these are parameters we really don't care about/want to learn!
chain_handler.ignore_plots(["LogL_systematic_xsec_cov", "Log", "LogL_systematic_osc_cov"])

# Now we add parameters we care about. Note this only need to be a substring of the full name
chain_handler.add_additional_plots(["sin2th", "delm2", "delta", "xsec"])

# The fitting label is special, it's the thing we want to train our network to predict
fitting_label = "LogL"
# We need to make sure the chain handler knows this exists, passing true means it is looking for some with that exact name
chain_handler.add_additional_plots(fitting_label, True)

# Finally we can do some cuts to get rid of things like burn-in
chain_handler.add_new_cuts(["LogL<30", "LogL_systematic_xsec_cov<1234", "step>10000"])

# Last step is convert the chain into a pandas dataframe
chain_handler.convert_ttree_to_array()

Let's quickly use the chain handler to make a plot of the trace and posterior!

In [ ]:
# We're just grab the first column of our pandas dataframe
parameter_vals = chain_handler.ttree_array.iloc[:,0]
# Grab the name from column label
param_label = chain_handler.ttree_array.columns[0]

# Setup figure
fig, (trace_ax, post_ax) = plt.subplots(nrows=1, ncols=2)

# Plot traces
trace_ax.plot(parameter_vals, linewidth=0.5, color='darkorange')

# Plot posterior
post_ax.hist(parameter_vals, bins=50, color='darkorange', alpha=0.5, orientation='horizontal', density=True)

# Merge the two axes
plt.setp(post_ax.get_yticklabels(), visible=False)
fig.subplots_adjust(wspace=.0)

# Set the axis labels
trace_ax.set_xlabel("Step")
post_ax.set_xlabel("Posterior Density")
trace_ax.set_ylabel(param_label)

plt.show()

## Machine learning
Okay now we've done some very basic file manipulation, it's time for some machine learning!
All algorithms use the same common `MLFactory` interface so let's go about configuring it!

In [ ]:
# First step we need to initialise the factory

from pathlib import Path


model_output=Path("../models/my_model")
# Make sure the model output directory exists
model_output.parent.mkdir(parents=True, exist_ok=True)

# The factory produces ML models, we need to pass it the chain handler and the fitting label to get started
ml_factory = MLFactory(chain_handler, fitting_label, f"{model_output}.pdf")

Now we need to define a model, for this demonstration we'll make a very simple neural network!

In [ ]:
# Finally we need to set the layers

# In the YAML format this would be simple but here we'll use some helper functions
def dense_layer(n_neurons: int, activation: str, regularize: bool = True):
    '''This function sets the properties of a dense layer'''
    return {'dense': {
        'units': n_neurons,
        'activation': activation,
        'kernel_regularizer': regularize
    }}
    
def batch_norm(momentum: float = 0.9):
    '''This function sets the properties of a batch norm layer'''
    return {'batchnorm': {
        'momentum': momentum
    }}
    
def dropout_layer(dropout_rate: float):
    '''This function sets the properties of a dropout layer'''
    return {'dropout': {
        'rate': dropout_rate
    }}


In [ ]:
layers = []

# Setup layers
layers.append(batch_norm(0.9))
layers.append(dense_layer(64, 'relu'))
layers.append(dense_layer(64, 'relu'))
layers.append(batch_norm(0.9))
layers.append(dense_layer(64, 'relu'))
layers.append(batch_norm(0.9))
layers.append(dense_layer(32, 'relu'))
layers.append(dense_layer(8, 'relu'))
layers.append(dense_layer(1, 'linear'))

In [ ]:
# We now want the actual neural network properties
# Usually you set this using a YAML file (see configs for some examples) but we'll do this manually for now
build_settings = {
    # Loss function
    'loss': 'mse',
    # Metrics to measure
    'metrics': ['mae', 'mse'],
    # Learning rate
    'learning_rate': 0.001,
}

# Settings used when the model is fitting
fit_settings = {
    'batch_size': 4096,
    'epochs': 100,
    'validation_split': 0.2,
}

In [ ]:
# Now we can make the model
ml_model = ml_factory.make_interface('TensorFlow', 'sequential', Layers=layers, FitSettings=fit_settings, BuildSettings=build_settings)

# Let's use 20% of the data for testing
ml_model.set_training_test_set(0.2)

In [ ]:
# Now we need to train our network
ml_model.train_model()

In [ ]:
# Now we need to test
ml_model.test_model()


In [ ]:
# Now we can save it
ml_model.save_model(f"{model_output}.keras")


# We need to save the scaling information
ml_model.save_scaler(str(model_output.parent.joinpath(Path("my_scaler.pkl"))))
print("Model saved to: ", model_output)
print("Scaler saved to: ", model_output.parent.joinpath(Path("my_scaler.pkl")))
